<a href="https://colab.research.google.com/github/tntnu/20242R0136COSE41600/blob/main/20242R0136COSE41600.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.0 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.

# 계층적 샘플링

In [ ]:
import os
import random
from collections import defaultdict

def get_files_by_behavior(folders):
    """
    각 행동(폴더)에서 PCD 파일을 분류하여 반환.
    """
    behavior_files = defaultdict(list)
    for folder in folders:
        behavior_name = os.path.basename(os.path.dirname(folder))  # 폴더 이름을 행동 이름으로 사용
        for file_name in os.listdir(folder):
            if file_name.endswith('.pcd'):
                file_path = os.path.join(folder, file_name)
                behavior_files[behavior_name].append(file_path)
    return behavior_files


In [ ]:
def stratified_sampling(behavior_files, behavior_ratios):
    """
    각 행동별로 설정된 비율로 데이터를 샘플링.
    """
    sampled_files = []
    for behavior, files in behavior_files.items():
        sample_ratio = behavior_ratios.get(behavior, 0.3)  # 비율이 지정되지 않은 경우 기본값 0.3
        sample_size = int(len(files) * sample_ratio)  # 행동별 샘플링 개수
        sampled_behavior_files = random.sample(files, min(sample_size, len(files)))  # 샘플링
        sampled_files.extend(sampled_behavior_files)  # 결과에 추가
        print(f"[{behavior}] 총 {len(files)}개 중 {len(sampled_behavior_files)}개 샘플링")
    return sampled_files


In [ ]:
# 행동별 PCD 파일이 저장된 폴더 리스트
folders = [
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/01_straight_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/02_straight_duck_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/03_straight_crawl/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/04_zigzag_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/05_straight_duck_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/06_straight_crawl/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/07_straight_walk/pcd"
]

In [ ]:
# 행동별 샘플링 비율 설정
behavior_ratios = {
    "01_straight_walk": 0.6,
    "02_straight_duck_walk": 0.5,
    "03_straight_crawl": 0.3,
    "04_zigzag_walk": 0.5,
    "05_straight_duck_walk": 0.5,
    "06_straight_crawl": 0.4,
    "07_straight_walk": 0.5
}

In [ ]:
# 1. 행동별 파일 리스트 가져오기
behavior_files = get_files_by_behavior(folders)

In [ ]:
# 각 행동별 파일 수 출력 (디버깅용)
for behavior, files in behavior_files.items():
    print(f"[{behavior}] 총 파일 수: {len(files)}")

[01_straight_walk] 총 파일 수: 288
[02_straight_duck_walk] 총 파일 수: 549
[03_straight_crawl] 총 파일 수: 1260
[04_zigzag_walk] 총 파일 수: 364
[05_straight_duck_walk] 총 파일 수: 577
[06_straight_crawl] 총 파일 수: 774
[07_straight_walk] 총 파일 수: 443


In [ ]:
# 2. 계층적 샘플링 수행
sampled_files = stratified_sampling(behavior_files, behavior_ratios)

print("\n샘플링된 총 파일 수:", len(sampled_files))

[01_straight_walk] 총 288개 중 172개 샘플링
[02_straight_duck_walk] 총 549개 중 274개 샘플링
[03_straight_crawl] 총 1260개 중 378개 샘플링
[04_zigzag_walk] 총 364개 중 182개 샘플링
[05_straight_duck_walk] 총 577개 중 288개 샘플링
[06_straight_crawl] 총 774개 중 309개 샘플링
[07_straight_walk] 총 443개 중 221개 샘플링

샘플링된 총 파일 수: 1824


In [ ]:
import shutil

In [ ]:
# 3. 샘플링된 파일 저장

def save_sampled_files(sampled_files, output_folder):
    """
    샘플링된 파일을 지정된 폴더에 저장.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_path in sampled_files:
        file_name = os.path.basename(file_path)
        shutil.copy(file_path, os.path.join(output_folder, file_name))

# 3. 샘플링된 파일 저장
output_folder = "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files"
save_sampled_files(sampled_files, output_folder)

# 저장된 파일 목록 출력 (디버깅용)
saved_files = os.listdir(output_folder)
print(f"저장된 파일 목록: {saved_files}")

저장된 파일 목록: ['pcd_000170.pcd', 'pcd_000039.pcd', 'pcd_000135.pcd', 'pcd_000196.pcd', 'pcd_000274.pcd', 'pcd_000242.pcd', 'pcd_000015.pcd', 'pcd_000052.pcd', 'pcd_000019.pcd', 'pcd_000081.pcd', 'pcd_000042.pcd', 'pcd_000145.pcd', 'pcd_000286.pcd', 'pcd_000080.pcd', 'pcd_000191.pcd', 'pcd_000204.pcd', 'pcd_000031.pcd', 'pcd_000139.pcd', 'pcd_000027.pcd', 'pcd_000213.pcd', 'pcd_000233.pcd', 'pcd_000004.pcd', 'pcd_000152.pcd', 'pcd_000120.pcd', 'pcd_000217.pcd', 'pcd_000026.pcd', 'pcd_000254.pcd', 'pcd_000011.pcd', 'pcd_000008.pcd', 'pcd_000216.pcd', 'pcd_000202.pcd', 'pcd_000024.pcd', 'pcd_000100.pcd', 'pcd_000230.pcd', 'pcd_000079.pcd', 'pcd_000261.pcd', 'pcd_000250.pcd', 'pcd_000198.pcd', 'pcd_000062.pcd', 'pcd_000193.pcd', 'pcd_000221.pcd', 'pcd_000157.pcd', 'pcd_000239.pcd', 'pcd_000126.pcd', 'pcd_000246.pcd', 'pcd_000143.pcd', 'pcd_000060.pcd', 'pcd_000041.pcd', 'pcd_000037.pcd', 'pcd_000092.pcd', 'pcd_000185.pcd', 'pcd_000272.pcd', 'pcd_000173.pcd', 'pcd_000211.pcd', 'pcd_000022.pcd'

# 최적의 voxel_size

In [ ]:
def voxel_down_sample(pcd, voxel_size):
    return pcd.voxel_down_sample(voxel_size)

def find_optimal_voxel_size(pcd):
    # 테스트할 voxel 크기 범위 정의
    voxel_sizes = np.arange(0.01, 0.1, 0.01)
    optimal_voxel_size = voxel_sizes[0]
    min_points = len(pcd.points)

    for voxel_size in voxel_sizes:
        downsampled_pcd = voxel_down_sample(pcd, voxel_size)
        if len(downsampled_pcd.points) < min_points:
            min_points = len(downsampled_pcd.points)
            optimal_voxel_size = voxel_size

    return optimal_voxel_size

In [ ]:
# PCD 파일이 있는 디렉토리 경로
pcd_directory = "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files"


In [ ]:
# 디렉토리 내 모든 PCD 파일 목록
pcd_files = [f for f in os.listdir(pcd_directory) if f.endswith('.pcd')]


In [ ]:
import numpy as np

# 각 PCD 파일 처리
for pcd_file in pcd_files:
    pcd_path = os.path.join(pcd_directory, pcd_file)
    pcd = o3d.io.read_point_cloud(pcd_path)

    # 다운샘플링을 위한 최적의 voxel 크기 찾기
    optimal_voxel_size = find_optimal_voxel_size(pcd)

    # 최적의 voxel 크기를 사용하여 포인트 클라우드 다운샘플링
    downsampled_pcd = voxel_down_sample(pcd, optimal_voxel_size)

    # 다운샘플링된 포인트 클라우드 저장
    downsampled_pcd_path = os.path.join(pcd_directory, f"downsampled_{pcd_file}")
    o3d.io.write_point_cloud(downsampled_pcd_path, downsampled_pcd)

    print(f"Processed {pcd_file} with optimal voxel size {optimal_voxel_size}. Downsampled file saved as {downsampled_pcd_path}.")

Processed pcd_000170.pcd with optimal voxel size 0.09. Downsampled file saved as /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files/downsampled_pcd_000170.pcd.
Processed pcd_000039.pcd with optimal voxel size 0.09. Downsampled file saved as /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files/downsampled_pcd_000039.pcd.
Processed pcd_000135.pcd with optimal voxel size 0.09. Downsampled file saved as /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files/downsampled_pcd_000135.pcd.
Processed pcd_000196.pcd with optimal voxel size 0.09. Downsampled file saved as /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files/downsampled_pcd_000196.pcd.
Processed pcd_000274.pcd with optimal voxel size 0.09. Downsampled file saved as /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files/dow

# 다운샘플링

In [3]:
import os
import open3d as o3d
import numpy as np

output_folder = "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files"


# 최적 voxel 크기
optimal_voxel_size = 0.09

# 결과를 저장할 디렉토리
downsampled_directory = os.path.join(output_folder, "downsampled_files")
os.makedirs(downsampled_directory, exist_ok=True)

# 디렉토리 내 모든 PCD 파일 목록
pcd_files = [f for f in os.listdir(output_folder) if f.endswith('.pcd')]

# PCD 파일 처리
for pcd_file in pcd_files:
    pcd_path = os.path.join(output_folder, pcd_file)

    # PCD 파일 로드
    pcd = o3d.io.read_point_cloud(pcd_path)

    if not pcd.has_points():
        print(f"Skipping {pcd_file} because it has no points.")
        continue

    # 다운샘플링
    downsampled_pcd = pcd.voxel_down_sample(optimal_voxel_size)

    # 다운샘플링된 PCD 파일 저장
    downsampled_pcd_path = os.path.join(downsampled_directory, f"downsampled_{pcd_file}")
    o3d.io.write_point_cloud(downsampled_pcd_path, downsampled_pcd)

    print(f"Processed {pcd_file}: {len(pcd.points)} -> {len(downsampled_pcd.points)} points. Saved to {downsampled_pcd_path}.")


Processed pcd_000532.pcd: 184320 -> 52673 points. Saved to /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files/downsampled_files/downsampled_pcd_000532.pcd.
Processed pcd_000369.pcd: 184320 -> 52768 points. Saved to /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files/downsampled_files/downsampled_pcd_000369.pcd.
Processed pcd_000046.pcd: 184320 -> 52069 points. Saved to /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files/downsampled_files/downsampled_pcd_000046.pcd.
Processed pcd_000397.pcd: 184320 -> 51460 points. Saved to /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files/downsampled_files/downsampled_pcd_000397.pcd.
Processed pcd_000476.pcd: 184320 -> 54761 points. Saved to /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files/downsampled_files/downsampled_pcd_000476.

In [6]:
# 다운샘플링된 파일이 저장된 디렉토리 경로
downsampled_directory = "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files/downsampled_files"

# 디렉토리 내 모든 PCD 파일 목록
downsampled_pcd_files = [f for f in os.listdir(downsampled_directory) if f.endswith('.pcd')]

# 파일 개수 출력
print(f"다운샘플링된 PCD 파일 개수: {len(downsampled_pcd_files)}")


다운샘플링된 PCD 파일 개수: 1614


In [4]:
o3d.visualization.draw_geometries([downsampled_pcd])

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


# .

In [ ]:
# 시각화에 필요한 라이브러리 불러오기
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

# pcd 파일 불러오기, 필요에 맞게 경로 수정
file_path = "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_tutorial/COSE416_HW1_tutorial/test_data/1727320101-665925967.pcd"
# PCD 파일 읽기
original_pcd = o3d.io.read_point_cloud(file_path)

In [ ]:
# Voxel Downsampling 수행
voxel_size = 0.5  # 필요에 따라 voxel 크기를 조정하세요. 0.2 -> 0.5로 수정
downsample_pcd = original_pcd.voxel_down_sample(voxel_size=voxel_size)

In [ ]:
# Radius Outlier Removal (ROR) 적용
cl, ind = downsample_pcd.remove_radius_outlier(nb_points=6, radius=1.2)
ror_pcd = downsample_pcd.select_by_index(ind)

In [ ]:
# RANSAC을 사용하여 평면 추정
plane_model, inliers = ror_pcd.segment_plane(distance_threshold=0.1,
                                             ransac_n=3,
                                             num_iterations=2000)

In [ ]:
# 도로에 속하지 않는 포인트 (outliers) 추출
final_point = ror_pcd.select_by_index(inliers, invert=True)

In [ ]:
# DBSCAN 클러스터링 적용
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(final_point.cluster_dbscan(eps=0.5, min_points=5, print_progress=True)) # eps = 0.3 -> 0.5, min_points = 10 -> 5 값 수정 후 클러스터링 성공

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 350


In [ ]:
# 노이즈 포인트는 검정색, 클러스터 포인트는 파란색으로 지정
colors = np.zeros((len(labels), 3))  # 기본 검정색 (노이즈)
colors[labels >= 0] = [0, 0, 1]  # 파란색으로 지정

final_point.colors = o3d.utility.Vector3dVector(colors)

In [ ]:
# 포인트 클라우드를 시각화하는 함수
def visualize_point_cloud(pcd, window_name="Point Cloud", point_size=1.0):
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name=window_name)
    vis.add_geometry(pcd)
    vis.get_render_option().point_size = point_size
    vis.run()
    vis.destroy_window()

In [ ]:
# 시각화 (포인트 크기를 원하는 크기로 조절 가능)
visualize_point_cloud(final_point, point_size=2.0)

# 세션 다운 됨

[Open3D WARNING] Failed to initialize GLFW


AttributeError: 'NoneType' object has no attribute 'point_size'